"Easy numbers":
- `1` has 2 segments
- `4` has 4 segments
- `7` has 3 segments
- `8` has 7 segments

"Hard numbers":
- `2` has 5 segments
- `3` has 5 segments
- `5` has 5 segments
- `0` has 6 segments
- `6` has 6 segments
- `9` has 6 segments

In [1]:
input_filename = 'input.txt'

In [2]:
from typing import List, Tuple

In [3]:
def normalize(pattern: str) -> str:
    """Sort the letters within each pattern to make my life easier in Part 2"""
    return "".join(sorted(pattern))


def parse_line(line: str) -> Tuple[List[str], List[str]]:
    pattern_group_raw, output_raw = line.strip().split('|')
    pattern_group = [normalize(pattern) for pattern in pattern_group_raw.split()]
    output = [normalize(pattern) for pattern in output_raw.split()]
    
    return pattern_group, output


with open(input_filename) as input_file:
    pattern_groups = []
    outputs = []
    
    for line in input_file.readlines():
        pattern_group, output = parse_line(line)
        pattern_groups.append(pattern_group)
        outputs.append(output)

# Part 1
Count number of `1`, `4`, `7`, and `8` digits in the outputs.

In [4]:
count = 0
for output in outputs:
    for digit in output:
        if len(digit) in (2, 4, 3, 7):
            count += 1

print(f"Answer to Part 1: {count}")

Answer to Part 1: 390


# Part 2
Deduce output value for each output, then sum them together.

In [5]:
# map number of segments to digit
easy_numbers_lengths = {2: 1, 4: 4, 3: 7, 7: 8}


def analyze_group(group: List[str]) -> Tuple[List[str], List[str]]:
    positions = ['' for _ in range(7)]
    digits = ['' for _ in range(10)]

    # First look at the easy digits: 1, 4, 7, 8 (they have unique numbers of segments)
    for pattern in group:
        if len(pattern) in easy_numbers_lengths:
            digits[easy_numbers_lengths[len(pattern)]] = pattern
        
    
    # Figure out position 0 by comparing digits 1 and 7
    for letter in digits[7]:
        if letter not in digits[1]:
            positions[0] = letter

    # Digits 0, 6, and 9 have 6 segments each, and we can figure out which is
    # digit 6 by comparing them against digit 1.
    # This will also tell us what positions 2 and 5 are.
    digits0or6or9 = [pattern for pattern in group if len(pattern) == 6]
    for idx, letter in enumerate(digits[1]):
        for digit0or6or9 in digits0or6or9:
            if letter not in digit0or6or9:
                # We've found digit 6
                digits[6] = digit0or6or9
                # We've also identified position 2 :)
                positions[2] = letter
                # And we can tell what position 5 is by process of elimination
                positions[5] = digits[1][idx-1]
                
    # Now we compare digits 0 and 9 against digit 4 to find out which is which.
    # This will tell us what position 3 is.
    digits0or9 = [pattern for pattern in group 
                  if len(pattern) == 6 and pattern != digits[6]]
    for letter in digits[4]:
        for idx, digit0or9 in enumerate(digits0or9):
            if letter not in digit0or9:
                # We've found digit 0
                digits[0] = digit0or9
                # We've also found digit 9 by process of elimination
                digits[9] = digits0or9[idx-1]
                # And we've identified position 3
                positions[3] = letter
                
    # We can find position 1 by inspecting digit 4 further.
    for letter in digits[4]:
        if letter not in positions:
            positions[1] = letter
            
    # We can find digit 3 by looking at digits with 5 segments
    # and checking if they have positions 2 and 5.
    for pattern in group:
        if len(pattern) == 5 and positions[2] in pattern and positions[5] in pattern:
            digits[3] = pattern
    
    # We can find position 6 by inspecting digit 3 further.
    for letter in digits[3]:
        if letter not in positions:
            positions[6] = letter
            
    # Positions left: 4
    # Digits left: 2, 5
    
    # We can find position 4 by process of elimination:
    for letter in 'abcdefg':
        if letter not in positions:
            positions[4] = letter

    # We can find digits 2 and 5 easily now.
    digits2or5 = [pattern for pattern in group 
                  if len(pattern) == 5 and pattern != digits[3]]
    digits[2] = [d for d in digits2or5 if positions[1] not in d][0]
    digits[5] = [d for d in digits2or5 if positions[1] in d][0]
    
    return digits, positions


def calculate_value(digits: List[str], output: List[str]) -> int:
    pattern_to_digit = {pattern: str(digit) for digit, pattern in enumerate(digits)}
    # because I'm lazy, I'm just going to join the string
    str_value = [pattern_to_digit[pattern] for pattern in output]
    return int("".join(str_value))



In [6]:
# Test simplest example

example_pattern_group = [normalize(pattern) for pattern in "acedgfb cdfbe gcdfa fbcad dab cefabd cdfgeb eafb cagedb ab".split()]
example_output = [normalize(pattern) for pattern in "cdfeb fcadb cdfeb cdbaf".split()]

digits, positions = analyze_group(example_pattern_group)
assert digits == ['abcdeg', 'ab', 'acdfg', 'abcdf', 'abef', 'bcdef', 'bcdefg', 'abd', 'abcdefg', 'abcdef']
assert positions == ['d', 'e', 'a', 'f', 'g', 'b', 'c']

assert calculate_value(digits, example_output) == 5353

In [7]:
# Let's do this for real now!
total = 0
for pattern_group, output in zip(pattern_groups, outputs):
    digits, _ = analyze_group(pattern_group)
    total += calculate_value(digits, output)

print(f"Answer to Part 2: {total}")

Answer to Part 2: 1011785


# Bonus
I decided to draw out the digits for "fun."

In [8]:
should_print = True

digit_to_segment_positions = {
    0: [0, 1, 2, 4, 5, 6],
    1: [2, 5],
    2: [0, 2, 3, 4, 6],
    3: [0, 2, 3, 5, 6],
    4: [1, 2, 3, 5],
    5: [0, 1, 3, 5, 6],
    6: [0, 1, 3, 4, 5, 6],
    7: [0, 2, 5],
    8: [0, 1, 2, 3, 4, 5, 6],
    9: [0, 1, 2, 3, 5, 6],
}

# Print out the digits to make sure I got the segment position mapping right
def print_digit(segments: List[int]):
    def get_symbol(pos: int) -> str:
        return "#" if pos in segments else " "

    # symbols
    s = [get_symbol(pos) for pos in range(7)]
    
    # Print segment 0
    print(f" {s[0]}{s[0]}{s[0]}{s[0]} ")
    # Print segments 1 and 2
    print(f"{s[1]}    {s[2]}\n{s[1]}    {s[2]}")
    # Print segment 3
    print(f" {s[3]}{s[3]}{s[3]}{s[3]} ")
    # Print segments 4 and 5
    print(f"{s[4]}    {s[5]}\n{s[4]}    {s[5]}")
    # Print segment 6
    print(f" {s[6]}{s[6]}{s[6]}{s[6]} ")
    print("\n")

if should_print:
    for segments in digit_to_segment_positions.values():
        print_digit(segments)

 #### 
#    #
#    #
      
#    #
#    #
 #### 


      
     #
     #
      
     #
     #
      


 #### 
     #
     #
 #### 
#     
#     
 #### 


 #### 
     #
     #
 #### 
     #
     #
 #### 


      
#    #
#    #
 #### 
     #
     #
      


 #### 
#     
#     
 #### 
     #
     #
 #### 


 #### 
#     
#     
 #### 
#    #
#    #
 #### 


 #### 
     #
     #
      
     #
     #
      


 #### 
#    #
#    #
 #### 
#    #
#    #
 #### 


 #### 
#    #
#    #
 #### 
     #
     #
 #### 


